In [ ]:
!git clone https://github.com/S4b3/ADM-HW3-Group1.git

Cloning into 'ADM-HW3-Group1'...
remote: Enumerating objects: 153084, done.
remote: Counting objects: 100% (153084/153084), done.
remote: Compressing objects: 100% (118228/118228), done.
remote: Total 153084 (delta 104460), reused 83465 (delta 34850), pack-reused 0
Receiving objects: 100% (153084/153084), 152.79 MiB | 24.53 MiB/s, done.
Resolving deltas: 100% (104460/104460), done.
Checking out files: 100% (57381/57381), done.


Useful libraries:

In [3]:
# Install BeautifulSoup, this will be needed to crawl the web
#!pip3 install beautifulsoup4
#!pip3 install tqdm
#!pip3 install nltk
!pip3 install tabulate

In [1]:
# Import asyncio, this will be needed to perform asynchronous operations
import os
import asyncio
# Importing multiprocessing to assign operat+ions to threadpools
import multiprocessing
from multiprocessing.dummy import Pool as ThreadPool
# Importing this to create necessary directories
import pathlib
from pathlib import Path
from datetime import datetime
from tqdm import tqdm
import nltk
from joblib import Parallel, delayed
import json 
import math
from collections import Counter
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None


nltk.download('stopwords')
nltk.download('punkt')


import functions

'''if this returns an error on tabulate, install it by running the cell above.'''

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Valentino\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Valentino\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'if this returns an error on tabulate, install it by running the cell above.'

In [2]:
'''
Defining the amount of cores available for the process to use. If this slows your machine too much, hardcode it. 
EX: AVAILABLE_CORES = 4
'''
AVAILABLE_CORES = multiprocessing.cpu_count()
print(AVAILABLE_CORES)

# Initializing ThreadPools 
pool = ThreadPool(AVAILABLE_CORES)

12


# 1.1. Get the list of animes   
We got all the anime urls and saved them in anime_urls.txt.


In [ ]:
def save_urls_to_txt(pages_of_urls):
    count = 0
    with open('./dataset/anime_urls.txt', 'w', encoding='utf-8') as txt_file :
        for page in pages_of_urls :
            for url in page :
                txt_file.write(url + os.linesep)
                count+=1
    print(f'written {count} lines')
    return True
            

## 1.2. Crawl animes

In [5]:
# Defining pages variables based on how many pages we want to retrieve
pages = [None] * 400
pages_num = range(0,400)

# Crawl Top Animes pages #
pool.map(lambda num : functions.fetch_page(num, pages), pages_num)   
# Parallel(n_jobs=AVAILABLE_CORES)(delayed(lambda num : functions.fetch_page(num, pages))(num) for num in tqdm(pages_num))
print("Done fetching the pages!")
print("Going to fetch urls")
# Scraping all URLs present in the crawled pages #
pool.map(lambda page : functions.fetch_urls_in_page(page, pages), pages)
#Parallel(n_jobs=AVAILABLE_CORES)(delayed(lambda page : functions.fetch_urls_in_page(page, pages))(page) for page in tqdm(pages))
print("Done fetching urls")

100%|████████████████████████████████████████████████████████████████████████████████████████| 400/400 [00:00<?, ?it/s]

Done fetching the pages!
Going to fetch urls


Done fetching urls


In [15]:
saved_file = save_urls_to_txt(pages)
if(saved_file):
    print("Saved all URLs in ./dataset/anime_urls.txt")
else :
    print("Couldn't save URLs in txt!")

written 19139 lines
Saved all URLs in ./dataset/anime_urls.txt


In [ ]:
# Fetch animes for every requested page
'''
Here we fetch and save animes in html files. 
Starting_page defines from which page you want to resume the process. (It works as an index)

EX: 
    to start from scratch:
        starting_page = 0
    if you want to start from the 10th page:
        starting_page = 9
    if you want to set 200 as an upper bound:
        last_page = 199   
'''
starting_page = 384
last_page = len(pages)
pages_to_process = pages[starting_page:]
for i in range(0, len(pages_to_process)) : 
    functions.fetch_animes_and_save_file(pages_to_process[i], starting_page+i+1, AVAILABLE_CORES)

In [3]:
with open('./dataset/page_1/article_1.html', 'r', encoding="utf-8") as f:
    url = functions.extract_url_from_html( f.read() )
    print(url)

https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood


## 1.3 Parse downloaded pages
At this point, you should have all the html documents about the animes of interest and you can start to extract the animes informations. The list of information we desire for each anime and their format is the following:

Anime Name (to save as animeTitle): String.  
Anime Type (to save as animeType): String.   
Number of episode (to save as animeNumEpisode): Integer.   
Release and End Dates of anime (to save as releaseDate and endDate): Convert both release and end date into datetime format.     
Number of members (to save as animeNumMembers): Integer.    
Score (to save as animeScore): Float.   
Users (to save as animeUsers): Integer    
Rank (to save as animeRank): Integer.    
Popularity (to save as animePopularity): Integer.    
Synopsis (to save as animeDescription): String.    
Related Anime (to save as animeRelated): Extract all the related animes, but only keep unique       values and those that have a hyperlink associated to them. List of strings.        
Characters (to save as animeCharacters): List of strings.         
Voices (to save as animeVoices): List of strings.     
Staff (to save as animeStaff): Include the staff  name and their responsibility/task in a list of lists.     

In [ ]:
# test code on just one single html:
functions.extract_informations_from_anime_html('./dataset/page_1/article_1.html')

In [4]:
# find all files into directories
matches = pathlib.Path("./dataset").glob("**/*.html")
# multiprocess parsing every html into a tsv
result = Parallel(n_jobs=AVAILABLE_CORES)(delayed(functions.extract_informations_from_anime_html)(path) for path in tqdm(matches))
#pool.map(extract_informations_from_anime_html, matches)

19124it [24:57, 12.77it/s]


# 2. Search Engine   
Now, we want to create two different Search Engines that, given as input a query, return the animes that match the query.   

First, you must pre-process all the information collected for each anime by:

**Removing stopwords**   
**Removing punctuation**   
**Stemming**   
**Anything else you think it's needed**   

For this purpose, you can use the nltk library.

In [4]:
functions.preprocess_tsv('./tsv_dataset/anime_11794.tsv')

In [4]:
tsv_matches = pathlib.Path("./tsv_dataset").glob("*.tsv")
#result = pool.map(functions.preprocess_tsv, tsv_matches)
for match in tsv_matches:
    functions.preprocess_tsv(match)

2.1. Conjunctive query   
For the first version of the search engine, we narrow our interest on the Synopsis of each anime. It means that you will evaluate queries only with respect to the anime's description.

2.1.1 Create your index!   
Before building the index,

- Create a file named vocabulary, in the format you prefer, that maps each word to an integer (term_id).

In [19]:
preprocessed_files = pathlib.Path("./preprocessed_dataset").glob("*.json")
functions.hydrate_vocabulary_from_files(preprocessed_files, AVAILABLE_CORES)

In [4]:
inverted_index_path = './indexes/synopsis_index.json'
vocabulary_path = './vocabulary.json'
Path("./indexes").mkdir(parents=True, exist_ok=True)
preprocessed_files = pathlib.Path("./preprocessed_dataset").glob("*.json")
functions.hydrate_synopsys_inverted_index_with_given_files_and_vocabulary(inverted_index_path, preprocessed_files, vocabulary_path, AVAILABLE_CORES)

1------------starting hydrating inverted index-------------
1------------inverted index not yet existant---------------
1------------reading vocabulary----------------------------
1------------proceeding to multiprocess inputs-------------
1------------finished to multiprocess inputs---------------
1------------finished hydrating inverted index-------------
1------------dumping inverted index -----------------------
1------------dumped inverted index to path-----------------
./indexes/synopsis_index.json


In [7]:
functions.take_simple_query_from_user()

Hi! Type in your query : 
 saiyan race
Results for "saiyan race" : 4 | elapsed time: 0.1091 seconds
| Title                                                   | Description   | Url                                                                                       |
|---------------------------------------------------------+---------------+-------------------------------------------------------------------------------------------|
| Dragon Ball Super: Broly                                | Forty-on..    | https://myanimelist.net/anime/36946/Dragon_Ball_Super__Broly                              |
| Dragon Ball Z Special 1: Tatta Hitori no Saishuu Kessen | Bardock,..    | https://myanimelist.net/anime/986/Dragon_Ball_Z_Special_1__Tatta_Hitori_no_Saishuu_Kessen |
| Dragon Ball Kai                                         | Five yea..    | https://myanimelist.net/anime/6033/Dragon_Ball_Kai                                        |
| Dragon Ball Z                                           | 

2.2) Conjunctive query & Ranking score<br />
For the second search engine, given a query, we want to get the top-k (the choice of k it's up to you!) documents related to the query. In particular:

Find all the documents that contains all the words in the query.   
Sort them by their similarity with the query.   
Return in output k documents, or all the documents with non-zero similarity with the query when the results are less than k. You must use a heap data structure (you can use Python libraries) for maintaining the top-k documents.   
To solve this task, you will have to use the **tfIdf** score, and the **Cosine similarity**. The field to consider it is still the synopsis. Let's see how.

2.2.1) Inverted index
Your second Inverted Index must be of this format:

In [5]:
inverted_index_path = './indexes/tf_idf_synopsis_index.json'
idf_source_index_path = './indexes/synopsis_index.json'
vocabulary_path = './vocabulary.json'
Path("./indexes").mkdir(parents=True, exist_ok=True)
preprocessed_files = pathlib.Path("./preprocessed_dataset").glob("*.json")

functions.hydrate_synopsys_inverted_index_with_given_files_and_vocabulary(
    inverted_index_path,
    preprocessed_files,
    vocabulary_path,
    AVAILABLE_CORES,
    True,
    19125,
    idf_source_index_path,
    )

1------------starting hydrating inverted index-------------
1------------inverted index not yet existant---------------
1------------reading vocabulary----------------------------
1------------proceeding to multiprocess inputs-------------
1------------finished to multiprocess inputs---------------
1------------finished hydrating inverted index-------------
1------------dumping inverted index -----------------------
1------------dumped inverted index to path-----------------
./indexes/tf_idf_synopsis_index.json


In [9]:
functions.take_top_k_of_query_from_user()

Hi! Type in your query : 
 dragon ball super
How many results would you like to retrieve?
 5
Results for "dragon ball super" : 7 | elapsed time: 1.425398 seconds
| Title                                                        | Description   | Url                                                                                           |   Similarity |
|--------------------------------------------------------------+---------------+-----------------------------------------------------------------------------------------------+--------------|
| Dragon Ball Super: Super Hero                                | Sequel m..    | https://myanimelist.net/anime/48903/Dragon_Ball_Super__Super_Hero                             |     0.638038 |
| Dragon Ball Z: The Real 4-D at Super Tenkaichi Budokai       | Dragon B..    | https://myanimelist.net/anime/42449/Dragon_Ball_Z__The_Real_4-D_at_Super_Tenkaichi_Budokai    |     0.611956 |
| Dragon Ball Super                                            | Seven

## Let's say that we are looking for an old dragon ball movie...

so we should look for *very few episodes*, a *quite old anime* with *kind of a good fanbase*...

In [6]:
functions.take_biased_query_from_user()

Hi! Type in your query : 
 dragon ball
How many results would you like to retrieve?
 5
You will now be asked to type in some preferences. 
Consider the range from -5 to 5, where -5 is the least, 5 is the max and 0 is indifferent.

How many episodes do you prefer?
 -5
Are you looking for a newer or an older anime?
 3
Are you looking for an anime with a large fanbase?
 3
Are you looking for an anime with a good score?
 4
Are you looking for a very popular anime?
 2
+----+--------------------------------------------------------------+--------------------+-----------------------------------------------------------------------------------------------+--------------+
|    | animeTitle                                                   | animeDescription   | url                                                                                           |   similarity |
|----+--------------------------------------------------------------+--------------------+--------------------------------------

##  5 ALGORITHMC QUESTION

Basically, given a list of integer and positive numbers (they are appointment's durations),we have to find the maximum sum of subsequence where the subsequence contains no element at adjacent positions.
We approached the problem using dynamic programming.
We create an auxiliary list $L_1$.

$L_1[i]$ represents the maximum possible sum for the sub-array staring from index ‘i’ and ending at index ‘N-1’, and $V$ is the initial input of length N.

For every index ‘i’, we have two choices:

1) Choose the current index:
   So that we will have 
   $L_1[i]$ = $V[i] + L_1[i-2]$


2) Skip the current index:
   $L_1[i]$ = $L_1[i-1]$
   
We will choose the path that maximizes our result:  $L_1[i]$ = max(  $V[i] + L_1[i-2]$, $L_1[i-1]$ )



$L_1[N-1]$ is the longest possible duration.

- Observation 1: In this way the computational time is $O(N)$
- Observation 2: We have to take care of the trivial cases, $N=0,1,2$

This is the implemtented program:

In [5]:
import numpy as np
n=int(input("How many appointment requests are there? --->"))
l = []
for i in range(1,n+1):
        x=int(input("Insert a value that correspond to the duration of the appointment  --->"))
        l.append(x)
        
print (l) #our list

How many appointment requests are there? --->6
Insert a value that correspond to the duration of the appointment  --->30
Insert a value that correspond to the duration of the appointment  --->40
Insert a value that correspond to the duration of the appointment  --->25
Insert a value that correspond to the duration of the appointment  --->50
Insert a value that correspond to the duration of the appointment  --->30
Insert a value that correspond to the duration of the appointment  --->20
[30, 40, 25, 50, 30, 20]


The function below is used to find the longest possible duration.

In [8]:
def max_seq(l):
    n=len(l)
    
    #basic cases

    if n == 0:

        return 0

    if n == 1:

        return l[0]

    if n == 2:

        return max(l[0], l[1])
    l_1 = [0]*n #new list
 

    
    l_1[0] = l[0]

    l_1[1] = max(l[0], l[1])

     

    

    for i in range(2, n):

        l_1[i] = max(l[i]+l_1[i-2], l_1[i-1])
    print("The longest possible duration is :" )
    print(l_1[-1])
 

    return l_1

In [9]:
max_seq(l)

The longest possible duration is :
110


[30, 40, 55, 90, 90, 110]

In this way we have found the maximum possible sum, but not the values that allow us to obtain it.
We initialize c with the maximum possible duration.
To extract the elements we  scroll through the auxiliary list starting from the end:
If the element in position i is greater than the next, it means that it is one of the elements that interest us.
So we append it in another auxiliary variable, and we subtract that value from our sum.
The boolean is needed to indicate the fact that if the element of index i has been taken from the list, the adjacent one must be skipped.


In [10]:
def extract_sub(l,l_1):
    l_1= l_1[::-1] #invert
    l= l[::-1]
    c= l_1[0]
    sub= []
    boolean = True
    N=len(l_1)
    for i in range(N):
        
        if i == N-1 and c > 0:
            sub.append(l[i])
            c-= l[i]
        elif not boolean:
            boolean = True
            continue
        elif  i < N-1 and l_1[i] > l_1[i+1] and boolean:
            boolean = False
            sub.append(l[i])
            c-= l[i]
    print("The selected appointments are:")
    return sub

In [11]:
extract_sub(l,max_seq(l))

The longest possible duration is :
110
The selected appointments are:


[20, 50, 40]